**Importing Libraries**

In [244]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier 
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, make_scorer, classification_report
import joblib
import sys

**Importing Dataset**

In [245]:
credit = pd.read_csv("C:/Users/Htet Myat/Desktop/Machinelearning/csvfile/credit_score.csv")

credit.head()

C:\Users\Htet Myat\AppData\Local\Temp\ipykernel_29664\1027220132.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  credit = pd.read_csv("C:/Users/Htet Myat/Desktop/Machinelearning/csvfile/credit_score.csv")


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [246]:
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [247]:
credit.isnull().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [248]:
credit.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

In [249]:
credit.value_counts

<bound method DataFrame.value_counts of             ID Customer_ID  ...     Monthly_Balance Credit_Score
0       0x1602   CUS_0xd40  ...  312.49408867943663         Good
1       0x1603   CUS_0xd40  ...  284.62916249607184         Good
2       0x1604   CUS_0xd40  ...   331.2098628537912         Good
3       0x1605   CUS_0xd40  ...  223.45130972736786         Good
4       0x1606   CUS_0xd40  ...  341.48923103222177         Good
...        ...         ...  ...                 ...          ...
99995  0x25fe9  CUS_0x942c  ...          479.866228         Poor
99996  0x25fea  CUS_0x942c  ...           496.65161         Poor
99997  0x25feb  CUS_0x942c  ...          516.809083         Poor
99998  0x25fec  CUS_0x942c  ...          319.164979     Standard
99999  0x25fed  CUS_0x942c  ...          393.673696         Poor

[100000 rows x 28 columns]>

**Dropping the unnecessary columns before training the model**

In [250]:
combined_cols = ['ID', 'Customer_ID', 'Name', 'SSN', 'Month', 'Type_of_Loan']

credit_score = credit.drop (columns=[col for col in combined_cols if col in credit.columns])
credit_score.head()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,1824.843333,3,4,3,4,3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,23,Scientist,19114.12,NaN,3,4,3,4,-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,-500,Scientist,19114.12,NaN,3,4,3,4,3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,23,Scientist,19114.12,NaN,3,4,3,4,5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,23,Scientist,19114.12,1824.843333,3,4,3,4,6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [251]:
#Using the function to replace in the cleaned strings after handling the various missing values in different forms in columns
def clean_dirty_numbers(x):
    if isinstance(x, str):
        # Remove '_', '!', '@', '#', '%', and strip whitespace
        x = x.replace('_', '').replace('!', '').replace('@', '').replace('#', '').replace('%', '').strip()
        if x == '':
            return np.nan
    try:
        return float(x)
    except ValueError:
        return np.nan

In [252]:
numeric_cols = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment', 
    'Changed_Credit_Limit', 'Outstanding_Debt', 'Amount_invested_monthly', 
    'Monthly_Balance', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Delay_from_due_date', 
    'Num_Credit_Inquiries', 'Credit_Utilization_Ratio', 'Total_EMI_per_month'
]

for col in numeric_cols:
    if col in credit_score.columns:
        credit_score[col] = credit_score[col].astype(str).apply(clean_dirty_numbers)


In [253]:
dirty_categorical_cols = ['Occupation', 'Payment_of_Min_Amount', 'Credit_Mix', 'Payment_Behaviour']


def clean_dirty_strings(series, col_name):
    series = series.astype(str).str.strip()
    series = series.replace(['_______', 'NM', 'nan', 'N/A', 'Unknown', '_'], np.nan)
    
    if col_name == 'Payment_Behaviour':
        def remove_junk(s):
            if pd.isna(s): return s
            s = str(s)
            if any(c.isdigit() or c in '!@#%.' for c in s):
                 return np.nan 
            return s
        series = series.apply(remove_junk)
    
    series = series.replace('', np.nan)
    
    return series

for col in dirty_categorical_cols:
    if col in credit_score.columns:
        credit_score[col] = clean_dirty_strings(credit_score[col], col)
        credit_score[col].replace("nan", np.nan)

for col in dirty_categorical_cols:
    if col in credit_score.columns:
        mode_val = credit_score[col].mode(dropna=True)[0]
        credit_score[col] = credit_score[col].fillna(mode_val)

if 'Credit_Score' in credit_score.columns:
    if credit_score['Credit_Score'].dtype == object or credit_score['Credit_Score'].isnull().any():
        if credit_score['Credit_Score'].notna().sum() > 0:
             mode_val = credit_score['Credit_Score'].astype(str).mode()[0]
             credit_score['Credit_Score'] = credit_score['Credit_Score'].fillna(mode_val)

**Handling the Payment_Behavior' categorial column**

In [254]:
valid_payment_values = [
    'High_spent_Small_value_payments',
    'High_spent_Medium_value_payments',
    'High_spent_Large_value_payments',
    'Low_spent_Small_value_payments',
    'Low_spent_Medium_value_payments',
    'Low_spent_Large_value_payments'
]

def clean_payment_behaviour(series):
    # Strip spaces
    series = series.astype(str).str.strip()

    # Replace invalid values with NaN
    series = series.apply(lambda x: x if x in valid_payment_values else np.nan)

    return series

credit_score['Payment_Behaviour'] = clean_payment_behaviour(credit_score['Payment_Behaviour'])

credit_score['Payment_Behaviour'] = credit_score['Payment_Behaviour'].fillna(
    credit_score['Payment_Behaviour'].mode()[0]
)


In [255]:
if 'Age' in credit_score.columns:
    credit_score.loc[credit_score['Age'] < 10, 'Age'] = np.nan

In [256]:
if 'Num_Credit_Card' in credit_score.columns:
    credit_score.loc[credit_score['Num_Credit_Card'] > 20, 'Num_Credit_Card'] = np.nan

if 'Interest_Rate' in credit_score.columns:
    credit_score.loc[credit_score['Interest_Rate'] >50, 'Interest_Rate'] = np.nan

if 'Num_of_Loan' in credit_score.columns:
    credit_score.loc[credit_score['Num_of_Loan'] >10, 'Num_of_Loan'] = np.nan

cols_to_fill = ['Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Occupation']

for col in cols_to_fill:
    if col in credit_score.columns:
        mode_value = credit_score[col].mode()[0]
        credit_score[col].fillna(mode_value, inplace=True)

C:\Users\Htet Myat\AppData\Local\Temp\ipykernel_29664\3168500251.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  credit_score[col].fillna(mode_value, inplace=True)


In [257]:
#Grouping the columns that cannot be negative values 
non_negative_cols = [
    'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 
    'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Num_of_Delayed_Payment', 
    'Num_Credit_Inquiries', 'Credit_Utilization_Ratio', 'Total_EMI_per_month',
    'Amount_invested_monthly', 'Outstanding_Debt', 'Delay_from_due_date'
]

for col in non_negative_cols:
    if col in credit_score.columns:
        credit_score.loc[credit_score[col] < 0, col] = np.nan
    
for col in credit_score.select_dtypes(include=[np.number]).columns:
    credit_score[col] = credit_score[col].fillna(credit_score[col].median())

In [276]:
def parse_credit_history(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, str):
        try:
            parts = x.split() 
            years = int(parts[0])
            months = int(parts[3])
            return (years * 12) + months
        except:
            return np.nan
    return x

if 'Credit_History_Age' in credit_score.columns:
    credit_score['Credit_History_Age'] = credit_score['Credit_History_Age'].apply(parse_credit_history)

In [259]:
#Numeric Imputation: Use Median (Must calculate median per column)
for col in credit_score.select_dtypes(include=[np.number]).columns:
    # Fills the NaNs in the current column (col) with the median of that specific column (df[col].median())
    credit_score[col] = credit_score[col].fillna(credit_score[col].median())


In [260]:
credit_score.head(50)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,4.0,Standard,809.98,26.822620,265.0,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
1,23.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,18.0,14.0,11.27,4.0,Good,809.98,31.944960,219.0,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good
2,33.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,3.0,7.0,9.40,4.0,Good,809.98,28.609352,267.0,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good
3,23.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,5.0,4.0,6.27,4.0,Good,809.98,31.377862,268.0,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good
4,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,6.0,14.0,11.27,4.0,Good,809.98,24.797347,269.0,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good
5,23.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,8.0,4.0,9.27,4.0,Good,809.98,27.262259,270.0,No,49.574949,62.430172,Low_spent_Small_value_payments,340.479212,Good
6,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,8.0,11.27,4.0,Good,809.98,22.537593,271.0,No,49.574949,178.344067,Low_spent_Small_value_payments,244.565317,Good
7,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,6.0,11.27,4.0,Good,809.98,23.933795,219.0,No,49.574949,24.785217,High_spent_Medium_value_payments,358.124168,Standard
8,28.0,Lawyer,34847.84,3037.986667,2.0,4.0,6.0,1.0,3.0,4.0,5.42,2.0,Good,605.03,24.464031,319.0,No,18.816215,104.291825,Low_spent_Small_value_payments,470.690627,Standard
9,28.0,Teacher,34847.84,3037.986667,2.0,4.0,6.0,1.0,7.0,1.0,7.42,2.0,Good,605.03,38.550848,320.0,No,18.816215,40.391238,High_spent_Large_value_payments,484.591214,Good


In [261]:
def clean_numeric_column(series):
    series = series.astype(str)
    series = pd.to_numeric(series, errors='coerce')
    series = series.replace(0, pd.NA)
    return series


def replace_outliers_with_median(series, upper_percentile=0.95):
    upper_limit = series.quantile(upper_percentile)
    median_val = series.median()
    return series.apply(lambda x: median_val if pd.notna(x) and (x > upper_limit) else x)


cols = ['Total_EMI_per_month', 'Amount_invested_monthly']

for col in cols:
    credit_score[col] = clean_numeric_column(credit_score[col])
    credit_score[col] = credit_score[col].fillna(credit_score[col].median())
    credit_score[col] = replace_outliers_with_median(credit_score[col], 0.95)



C:\Users\Htet Myat\AppData\Local\Temp\ipykernel_29664\351537016.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  credit_score[col] = credit_score[col].fillna(credit_score[col].median())
C:\Users\Htet Myat\AppData\Local\Temp\ipykernel_29664\351537016.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  credit_score[col] = credit_score[col].fillna(credit_score[col].median())


In [262]:
credit_score.head(50)

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,4.0,Standard,809.98,26.822620,265.0,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
1,23.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,18.0,14.0,11.27,4.0,Good,809.98,31.944960,219.0,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good
2,33.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,3.0,7.0,9.40,4.0,Good,809.98,28.609352,267.0,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good
3,23.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,5.0,4.0,6.27,4.0,Good,809.98,31.377862,268.0,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good
4,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,6.0,14.0,11.27,4.0,Good,809.98,24.797347,269.0,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good
5,23.0,Scientist,19114.12,3093.745000,3.0,4.0,3.0,4.0,8.0,4.0,9.27,4.0,Good,809.98,27.262259,270.0,No,49.574949,62.430172,Low_spent_Small_value_payments,340.479212,Good
6,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,8.0,11.27,4.0,Good,809.98,22.537593,271.0,No,49.574949,178.344067,Low_spent_Small_value_payments,244.565317,Good
7,23.0,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,6.0,11.27,4.0,Good,809.98,23.933795,219.0,No,49.574949,24.785217,High_spent_Medium_value_payments,358.124168,Standard
8,28.0,Lawyer,34847.84,3037.986667,2.0,4.0,6.0,1.0,3.0,4.0,5.42,2.0,Good,605.03,24.464031,319.0,No,18.816215,104.291825,Low_spent_Small_value_payments,470.690627,Standard
9,28.0,Teacher,34847.84,3037.986667,2.0,4.0,6.0,1.0,7.0,1.0,7.42,2.0,Good,605.03,38.550848,320.0,No,18.816215,40.391238,High_spent_Large_value_payments,484.591214,Good


In [263]:
credit_score.isnull().sum()

Age                         0
Occupation                  0
Annual_Income               0
Monthly_Inhand_Salary       0
Num_Bank_Accounts           0
Num_Credit_Card             0
Interest_Rate               0
Num_of_Loan                 0
Delay_from_due_date         0
Num_of_Delayed_Payment      0
Changed_Credit_Limit        0
Num_Credit_Inquiries        0
Credit_Mix                  0
Outstanding_Debt            0
Credit_Utilization_Ratio    0
Credit_History_Age          0
Payment_of_Min_Amount       0
Total_EMI_per_month         0
Amount_invested_monthly     0
Payment_Behaviour           0
Monthly_Balance             0
Credit_Score                0
dtype: int64

In [264]:
encoders = {}
categorical_cols = ['Occupation', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour', 'Credit_Score']
target_col = 'Credit_Score'

for col in categorical_cols:
    if col in credit_score.columns:
        le = LabelEncoder()
        credit_score[col] = le.fit_transform(credit_score[col].astype(str))
        encoders[col] = le

In [265]:
credit_score.head()

,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,3.0,7.0,11.27,4.0,2,809.98,26.822620,265.0,0,49.574949,80.415295,2,312.494089,0
1,23.0,12,19114.12,3093.745000,3.0,4.0,3.0,4.0,18.0,14.0,11.27,4.0,1,809.98,31.944960,219.0,0,49.574949,118.280222,3,284.629162,0
2,33.0,12,19114.12,3093.745000,3.0,4.0,3.0,4.0,3.0,7.0,9.40,4.0,1,809.98,28.609352,267.0,0,49.574949,81.699521,4,331.209863,0
3,23.0,12,19114.12,3093.745000,3.0,4.0,3.0,4.0,5.0,4.0,6.27,4.0,1,809.98,31.377862,268.0,0,49.574949,199.458074,5,223.451310,0
4,23.0,12,19114.12,1824.843333,3.0,4.0,3.0,4.0,6.0,14.0,11.27,4.0,1,809.98,24.797347,269.0,0,49.574949,41.420153,1,341.489231,0


In [266]:
X = credit_score.drop(columns=[target_col])
y = credit_score[target_col]
target_encoder = encoders[target_col]

all_unique_labels = ['Good', 'Standard', 'Poor']

target_encoder.fit(all_unique_labels)
poor_label = target_encoder.transform(['Poor'])[0]
scorer = make_scorer(recall_score, average=None, labels=[poor_label])

In [267]:
if X.empty or len(X.columns) == 0:
    raise ValueError
if y.empty:
    raise ValueError

In [268]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [269]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)


In [270]:
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15)
model.fit(X_train_scaled, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,15
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [271]:
y_pred = model.predict(X_test_scaled)

In [272]:
accuracy = accuracy_score(y_pred, y_test)
accuracy

0.74595

In [273]:
cm = confusion_matrix(y_test, y_pred)
class_labels = target_encoder.classes_

cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)
print(cm_df)

          Good  Poor  Standard
Good      2396    25      1106
Poor       315  4277      1282
Standard  1136  1217      8246


In [274]:
print(classification_report(y_test, y_pred, target_names=class_labels))

              precision    recall  f1-score   support

        Good       0.62      0.68      0.65      3527
        Poor       0.77      0.73      0.75      5874
    Standard       0.78      0.78      0.78     10599

    accuracy                           0.75     20000
   macro avg       0.72      0.73      0.73     20000
weighted avg       0.75      0.75      0.75     20000



In [283]:
import joblib
joblib.dump(model, 'model.pkl', compress=9)

print("saved the model pkl")

saved the model pkl
